<a href="https://colab.research.google.com/github/Zakibrahmi/AgentAI/blob/main/LangChainAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agents in LangChain

**Agent Architecture:**

As an agent architecture, ReAct will be used.  **ReAct**, for  **Re**asoning and **Act**ing was proposed by researchers at Google AI in a paper titled [ReAct: Synergizing Reasoning and Acting in Language Models](https://par.nsf.gov/biblio/10451467).

 ReAct defines how the LLM **reasons** about tasks, selects **actions**, and utilizes **tools** to achieve goals.

**LLM-powered agents framwwork: LangChain**

LangChain a framework for developing applications powered by LLM. It provides tools to manage prompts, connect to external data sources, and create *chains* of calls to LLMs or other utilities.


**Goal of the tutorial:**

The gaol is to  calculate the length of one side of a roof. If you know the lengths of two beams that support the roof at a right angle, you can use their lengths to calculate the length of the roof using the hypotenuse formula below.



# Let practice !!

**Install Librairies:** langchain, langgraph

In [15]:
!pip install langchain_openai
!pip install langgraph langgraph-prebuilt
!pip install openai


**Import Libraries**

In [16]:
# Module imports
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
import math


Initializes the LLM that will power the agent. Think of the LLM as the "brain" of the operation, responsible for understanding your requests and figuring out how to use the available tools to answer them. For this task, **gpt-4o-mini** a variant of  GPT-4 is used.

In [17]:
# LLM Setup
model = ChatOpenAI(openai_api_key="<you key>", model="gpt-4o-mini")

**Creat a tool** to be used by the agent. The tool is a funciton to calaculate the length of the  the hypotenuse of a right-angled triangle.
As you see, the *hypotenuse_length* function is designed to be used as a tool within the *LangChain agent framework*, which often works with *text-based* interactions, receiving natural language queries and providing text-based responses.

In [18]:
# Define this math function as a tool
@tool
def hypotenuse_length(input: str) -> float:
    """Calculates the length of the hypotenuse of a right-angled triangle given the lengths of the other two sides."""

    # Split the input string to get the lengths of the triangle
    sides = input.split(',')

    # Convert the input values to floats, removing extra spaces
    a = float(sides[0].strip())
    b = float(sides[1].strip())

    # Square each of the values, add them together, and find the square root
    return math.sqrt(a**2 + b**2)

# Run Agent
To invoke agent there is to path:
1. Agent execution flow
2. Chain

**Agent execution flow**

In [21]:
import time
from openai import RateLimitError # Import RateLimitError from openai.error


# Create a list variable and pass in your tool
tools = [hypotenuse_length]

# Create a query using natural language
query = "What is the hypotenuse length of a triangle with side lengths of 10 and 12?"

# Pass in the hypotenuse length tool and create the agent
app = create_react_agent(model, tools)

# Invoke the agent and print the response
try:
    response = app.invoke({"messages": [("human", query)]})
    print(response['messages'][-1].content)
except RateLimitError as e:
    print(f"Rate limit error: {e}")
    print("Retrying in 30 seconds...")
    time.sleep(30)  # Wait for 30 seconds before retrying
    response = app.invoke({"messages": [("human", query)]})
    print(response['messages'][-1].content)

Rate limit error: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Retrying in 30 seconds...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

*Sorry fo the exception, I don't have quota* :)))

**Chain**:
Here, I will create a prompt template to guide the LLM using a Chain-of-Thought (CoT) approach, and leverage LangChain's LLMChain for a more structured and modular workflow.

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

template = """You are a helpful assistant that can calculate the hypotenuse of a right-angled triangle.
You have access to a tool that can do the calculation for you.

Question: {question}

Think step-by-step about how to solve the problem. If you need to use the tool, mention its name (hypotenuse_length) in your reasoning.

Answer:"""

prompt = PromptTemplate(
    input_variables=["question"],
    template=template,
)

# Create the LLMChain
llm_chain = LLMChain(llm=model, prompt=prompt)

# Create a list variable and pass in your tool
tools = [hypotenuse_length]

# Pass in the hypotenuse length tool and create the agent
app = create_react_agent(model, tools)

# Create a query using natural language
query = "What is the hypotenuse length of a triangle with side lengths of 10 and 12?"

# Invoke the LLMChain to get the initial response
llm_response = llm_chain.run(query)

# If the LLM needs to use the tool, it should mention it in the response.
# Here, we're assuming a simple case where the LLM either directly answers or suggests using the tool.
if "hypotenuse_length" in llm_response.lower():  # Check if the tool name is mentioned
    # Invoke the agent to use the tool
    response = app.invoke({"messages": [("human", query)]})
    final_answer = response['messages'][-1].content
else:
    final_answer = llm_response  # Use the LLM's direct answer

print(final_answer)